In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-28 16:20:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.67MB/s    in 0.6s    

2021-03-28 16:20:56 (1.67 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [57]:
from pyspark.sql.functions import to_date
#Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [58]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [59]:
df_morethanhalf=vine_df.filter(vine_df.helpful_votes/vine_df.total_votes >= 0.5)


In [77]:
unpaid_df = df_morethanhalf.filter(df_morethanhalf.vine == 'N')
unpaid_review = unpaid_df.count()

unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23H79DHOZTYAU|          1|            1|          1|   N|                Y|
|R3AACSKX1EAHUY|          5|            1|          1|   N|                Y|
|R1FJWU0IJ4KUT4|          1|            5|          5|   N|                Y|
|R23HC6NMA3HBPS|          5|            1|          1|   N|                N|
| R8HD4KOH8L0A6|          5|            1|          2|   N|                Y|
|R3DCTXDTB028VH|          5|            3|          4|   N|                Y|
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2NCZ6CIHAWH28|          5|            1|          1|   N|                Y|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
| RXJ4RXVZNQQMK|          1|            1|          1|   N|     

In [76]:
paid_df = df_morethanhalf.filter(df_morethanhalf.vine == 'Y')
paid_review = paid_df.count()
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RWKQQPG100CQ4|          4|            2|          4|   Y|                N|
|R1R0J8S1VGALUY|          1|            1|          2|   Y|                N|
|R28165EG0N9CSN|          4|            1|          2|   Y|                N|
|R1CH0ZXGIMVEFC|          5|            1|          2|   Y|                N|
|R2BFEJHEKH4YWZ|          5|            3|          4|   Y|                N|
| RYIG0BYEZP7WM|          2|            1|          1|   Y|                N|
|R20FZQEZM1277P|          4|            2|          4|   Y|                N|
| RSAY2UB0IHC0L|          4|            4|          6|   Y|                N|
| RQKQVDDOKW5MU|          5|            4|          5|   Y|                N|
|R379LKZSA2PFOZ|          4|            1|          2|   Y|     

In [83]:
paid_review

1757

In [84]:
unpaid_review

572411

In [68]:
count_df = vine_program_df.groupby("star_rating").agg({"star_rating":"count"}).show()


+-----------+------------------+
|star_rating|count(star_rating)|
+-----------+------------------+
|          1|             77775|
|          3|             55109|
|          5|            291135|
|          4|            110841|
|          2|             37551|
+-----------+------------------+



In [69]:
df_morethanhalf.agg({"total_votes":"count"}).show()

+------------------+
|count(total_votes)|
+------------------+
|            574168|
+------------------+



In [79]:
paid_5star = paid_df.filter(paid_df.star_rating == 5).count()
paid_5star/paid_review

0.398406374501992

In [80]:
unpaid_5star = unpaid_df.filter(paid_df.star_rating == 5).count()
unpaid_5star/unpaid_review

0.5086118191299608